## Opgave 3.1

In [ ]:
import numpy as np
import math
import time
from functools import partial
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar, root_scalar
import pickle

# -------------------------
# Calibration and constants
# -------------------------
ell_max = 16.0              # maximum hours (ℓ)
w = 1.0                     # wage
tau_star = 0.1384375        # τ* provided by the user
zeta_star = -0.5354430379746835  # ζ* provided by the user
kappa = 9.0                 # top-tax cutoff κ
omega = 0.2                 # additional top tax ω
eps = 1.0                   # ϵ in utility (default baseline)
nu = 0.015                  # ν in utility (baseline)
l_plot_min = 0.5            # lower bound for plotting as requested in assignment
n_grid = 400                # resolution for continuous plots

# -------------------------
# Worker class
# -------------------------
class Worker:
    """
    Minimal Worker class to evaluate income, utility, and FOC expressions.

    Attributes
    ----------
    pi : float
        Individual productivity.
    w, tau, zeta, kappa, omega, nu, eps, ell_max : floats
        Model parameters (see top of script).

    Methods
    -------
    income(l)
        Returns post-tax income y(l) under the top tax.
    utility(l)
        Returns utility U = log(c) - nu * l^(1+eps)/(1+eps) with c = income(l).
        Returns -np.inf if consumption c <= 0 to mark infeasible choices.
    phi_pre(l)
        Returns FOC phi when marginal tax is tau (applies for l < l_k).
    phi_post(l)
        Returns FOC phi when marginal tax is tau+omega (applies for l > l_k).
    """
    def __init__(self, pi, w=w, tau=tau_star, zeta=zeta_star,
                 kappa=kappa, omega=omega, nu=nu, eps=eps, ell_max=ell_max):
        self.pi = float(pi)
        self.w = float(w)
        self.tau = float(tau)
        self.zeta = float(zeta)
        self.kappa = float(kappa)
        self.omega = float(omega)
        self.nu = float(nu)
        self.eps = float(eps)
        self.ell_max = float(ell_max)
        # kink hours: l_k = kappa / (w * pi)
        self.l_k = self.kappa / (self.w * self.pi)

    def income(self, l):
        """
        Compute post-tax income under top tax:
        y = (1 - tau) * w * p * l - omega * max(w * p * l - kappa, 0) - zeta
        Accepts scalar or numpy array l.
        """
        gross = self.w * self.pi * l
        # top tax part applies only to earnings above kappa
        top_part = np.maximum(gross - self.kappa, 0.0)
        y = (1.0 - self.tau) * gross - self.omega * top_part - self.zeta
        return y

    def utility(self, l):
        """
        Return utility U(l) = log(c) - nu * l^(1+eps)/(1+eps), where c = income(l).
        If income/l consumption is non-positive, return -np.inf.
        The method handles scalar or numpy-array input.
        """
        c = self.income(l)
        # Vectorized handling
        if isinstance(c, np.ndarray):
            u = np.full_like(c, -np.inf, dtype=float)
            feasible = c > 0
            # compute utility only for feasible entries
            u[feasible] = np.log(c[feasible]) - self.nu * (l[feasible] ** (1.0 + self.eps)) / (1.0 + self.eps)
            return u
        else:
            # scalar case
            if c <= 0:
                return -np.inf
            return math.log(c) - self.nu * (l ** (1.0 + self.eps)) / (1.0 + self.eps)

    def phi_pre(self, l):
        """
        FOC phi in the pre-kink region (marginal tax = tau):
        phi = (1 - tau) * w * p / c - nu * l^eps
        Returns -np.inf for infeasible c (non-positive).
        Works vectorized or scalar.
        """
        c = self.income(l)
        if isinstance(c, np.ndarray):
            phi_vals = np.full_like(c, -np.inf, dtype=float)
            feasible = c > 0
            phi_vals[feasible] = (1.0 - self.tau) * self.w * self.pi / c[feasible] - self.nu * (l[feasible] ** self.eps)
            return phi_vals
        else:
            if c <= 0:
                return -np.inf
            return (1.0 - self.tau) * self.w * self.pi / c - self.nu * (l ** self.eps)

    def phi_post(self, l):
        """
        FOC phi in the post-kink region (marginal tax = tau + omega):
        phi = (1 - tau - omega) * w * p / c - nu * l^eps
        Returns -np.inf for infeasible c (non-positive).
        Works vectorized or scalar.
        """
        c = self.income(l)
        if isinstance(c, np.ndarray):
            phi_vals = np.full_like(c, -np.inf, dtype=float)
            feasible = c > 0
            phi_vals[feasible] = (1.0 - self.tau - self.omega) * self.w * self.pi / c[feasible] - self.nu * (l[feasible] ** self.eps)
            return phi_vals
        else:
            if c <= 0:
                return -np.inf
            return (1.0 - self.tau - self.omega) * self.w * self.pi / c - self.nu * (l ** self.eps)

# -------------------------
# Solver utilities
# -------------------------
def find_opt_numerical(worker, bounds=(0.0, ell_max)):
    """
    Find optimal labor choice using a numerical optimizer.
    We maximize utility by minimizing negative utility over the given bounds.
    Returns:
        l_opt (float), u_opt (float), elapsed_time (float), result_object (scipy OptimizeResult)
    """
    t0 = time.perf_counter()

    def neg_u(l):
        # objective for minimizer; large positive value for infeasible choices
        u = worker.utility(l)
        if u == -np.inf:
            return 1e12
        return -u

    # use bounded scalar minimization
    res = minimize_scalar(neg_u, bounds=bounds, method='bounded', options={'xatol': 1e-9})
    t1 = time.perf_counter()
    l_opt = res.x
    u_opt = -res.fun if res.success else worker.utility(l_opt)
    return l_opt, u_opt, (t1 - t0), res

def find_opt_foc_four_step(worker):
    """
    Implement the four-step FOC approach described in the assignment. Steps:
    1) Solve phi_pre(l)=0 on (0, l_k) -> candidate l_b (if exists)
    2) Evaluate utility at kink l_k -> candidate l_k
    3) Solve phi_post(l)=0 on (l_k, ell_max) -> candidate l_a (if exists)
    4) Choose the candidate with the highest utility (ignore infeasible ones).
    Returns:
        l_choice (float), u_choice (float), elapsed_time (float), candidates (list), chosen_detail (tuple)
    candidates is a list of tuples (label, l, utility) where label is 'b', 'k', or 'a'.
    """
    t0 = time.perf_counter()
    candidates = []
    l_k = worker.l_k

    # Step 1: pre-kink interior solution (0, l_k) using root-finding on phi_pre
    if l_k > 1e-12:
        # scan a grid to detect sign changes (bracketing for reliable root finding)
        a_scan = max(1e-8, 1e-6)
        b_scan = min(l_k * 0.9999, worker.ell_max)
        if b_scan > a_scan:
            grid = np.linspace(a_scan, b_scan, 300)
            phi_vals = worker.phi_pre(grid)
            # find indices where sign changes occur (exclude -inf entries)
            finite_mask = np.isfinite(phi_vals)
            idx = np.where(finite_mask[:-1] & finite_mask[1:] & (np.sign(phi_vals[:-1]) * np.sign(phi_vals[1:]) < 0))[0]
            found = False
            for i in idx:
                a = grid[i]; b = grid[i+1]
                try:
                    sol = root_scalar(lambda x: worker.phi_pre(x), bracket=[a,b], method='bisect')
                    if sol.converged and 0 <= sol.root <= worker.ell_max:
                        l_b = sol.root
                        u_b = worker.utility(l_b)
                        candidates.append(('b', l_b, u_b))
                        found = True
                        break
                except Exception:
                    # if bisect fails for some reason, continue to next bracket
                    continue
            # If no sign change found, no interior root in (0,l_k)
    # Step 2: kink point candidate
    if 0.0 <= l_k <= worker.ell_max:
        u_k = worker.utility(l_k)
        candidates.append(('k', l_k, u_k))

    # Step 3: post-kink interior solution (l_k, ell_max)
    if l_k < worker.ell_max - 1e-12:
        a_scan = max(l_k * 1.0001, 1e-6)
        b_scan = worker.ell_max
        if b_scan > a_scan:
            grid = np.linspace(a_scan, b_scan, 400)
            phi_vals = worker.phi_post(grid)
            finite_mask = np.isfinite(phi_vals)
            idx = np.where(finite_mask[:-1] & finite_mask[1:] & (np.sign(phi_vals[:-1]) * np.sign(phi_vals[1:]) < 0))[0]
            for i in idx:
                a = grid[i]; b = grid[i+1]
                try:
                    sol = root_scalar(lambda x: worker.phi_post(x), bracket=[a,b], method='bisect')
                    if sol.converged and 0 <= sol.root <= worker.ell_max:
                        l_a = sol.root
                        u_a = worker.utility(l_a)
                        candidates.append(('a', l_a, u_a))
                        break
                except Exception:
                    continue

    # Step 4: choose best candidate (highest utility among candidates with finite utility)
    finite_candidates = [c for c in candidates if c[2] != -np.inf and not np.isnan(c[2])]
    if len(finite_candidates) == 0:
        # if no feasible candidate, choose l=0 (feasible since zeta <= 0)
        l_choice = 0.0
        u_choice = worker.utility(0.0)
        chosen_detail = ('none', l_choice, u_choice)
    else:
        best = max(finite_candidates, key=lambda c: c[2])
        l_choice = best[1]
        u_choice = best[2]
        chosen_detail = best

    t1 = time.perf_counter()
    return l_choice, u_choice, (t1 - t0), candidates, chosen_detail

# -------------------------
# Main analysis for Task 3.1
# -------------------------
def task_3_1_analysis(pi_values, save_path='/mnt/data/task3_1_results.pkl', show_plots=True):
    """
    Run Task 3.1 analysis for a list of pi values.
    For each pi:
      - create Worker instance
      - compute utility on [l_plot_min, ell_max]
      - compute phi_pre on [l_plot_min, l_k] and phi_post on [l_k, ell_max]
      - compute numerical optimum and four-step FOC solution
      - produce a combined figure (4 panels) summarising results
    Results are saved (pickled) to save_path for reproducibility.
    """
    results = {}
    l_full_grid = np.linspace(l_plot_min, ell_max, n_grid)

    for pi in pi_values:
        # create worker
        worker = Worker(pi=pi)

        # compute utility across grid (use vectorised evaluation)
        U_vals = worker.utility(l_full_grid)

        # define pre-kink and post-kink grids for phi plotting
        # ensure we follow assignment instruction to plot pre-kink up to min(kink, ell)
        l_k = worker.l_k
        grid_pre = np.linspace(l_plot_min, min(l_k, ell_max), 200) if min(l_k, ell_max) > l_plot_min else np.array([])
        phi_pre_vals = worker.phi_pre(grid_pre) if grid_pre.size > 0 else np.array([])

        grid_post = np.linspace(max(l_k, l_plot_min), ell_max, 200)
        phi_post_vals = worker.phi_post(grid_post)

        # numerical optimization
        l_opt_num, u_opt_num, t_num, res_obj = find_opt_numerical(worker, bounds=(0.0, worker.ell_max))

        # four-step FOC approach
        l_opt_foc, u_opt_foc, t_foc, candidates, chosen_detail = find_opt_foc_four_step(worker)

        # store results for this pi
        results[pi] = {
            'worker': worker,
            'l_full_grid': l_full_grid,
            'U_vals': U_vals,
            'grid_pre': grid_pre,
            'phi_pre_vals': phi_pre_vals,
            'grid_post': grid_post,
            'phi_post_vals': phi_post_vals,
            'l_opt_num': l_opt_num,
            'u_opt_num': u_opt_num,
            't_num': t_num,
            'res_obj': res_obj,
            'l_opt_foc': l_opt_foc,
            'u_opt_foc': u_opt_foc,
            't_foc': t_foc,
            'candidates': candidates,
            'chosen_detail': chosen_detail
        }

        # Plot the combined figure if requested
        if show_plots:
            fig, axes = plt.subplots(2, 2, figsize=(12, 9))
            fig.suptitle(f"Task 3.1 - pi = {pi:.3f}   (tau={tau_star}, zeta={zeta_star}, kappa={kappa}, omega={omega})")

            # Top-left: Utility over full l grid
            ax = axes[0, 0]
            ax.plot(l_full_grid, U_vals)
            ax.set_xlabel('Hours ℓ')
            ax.set_ylabel('Utility U(π, c(ℓ), ℓ)')
            ax.set_title('Utility as function of hours')
            # mark numerical and foc optima
            ax.axvline(l_opt_num, linestyle='--', label='numerical l*')
            ax.axvline(l_opt_foc, linestyle=':', label='FOC l*')
            # annotate values close to each line
            ax.annotate(f"num l*={l_opt_num:.4f}", xy=(l_opt_num, u_opt_num), xytext=(l_opt_num + 0.3, u_opt_num - 0.02))
            ax.annotate(f"foc l*={l_opt_foc:.4f}", xy=(l_opt_foc, u_opt_foc), xytext=(l_opt_foc + 0.3, u_opt_foc - 0.04))

            # Top-right: phi pre-kink
            ax = axes[0, 1]
            if grid_pre.size > 0:
                ax.plot(grid_pre, phi_pre_vals)
                ax.set_xlabel('Hours ℓ')
                ax.set_ylabel('phi_pre(ℓ)')
                ax.set_title('FOC phi (pre-kink)')
                ax.axhline(0, linestyle='--')
                ax.axvline(worker.l_k, linestyle=':', label='kink ℓ_k')
            else:
                # If no pre-kink region on plotting domain, show message
                ax.text(0.05, 0.5, "No pre-kink region within plotting domain", transform=ax.transAxes)
                ax.set_title('FOC phi (pre-kink)')

            # Bottom-left: phi post-kink
            ax = axes[1, 0]
            ax.plot(grid_post, phi_post_vals)
            ax.set_xlabel('Hours ℓ')
            ax.set_ylabel('phi_post(ℓ)')
            ax.set_title('FOC phi (post-kink)')
            ax.axhline(0, linestyle='--')
            ax.axvline(worker.l_k, linestyle=':')

            # Bottom-right: textual summary including timings and candidates
            ax = axes[1, 1]
            ax.axis('off')
            lines = []
            lines.append(f"pi = {pi:.3f}")
            lines.append(f"kink hours ℓ_k = {worker.l_k:.6f}")
            lines.append("Numerical optimizer:")
            lines.append(f"  l* = {l_opt_num:.6f}, U = {u_opt_num:.6f}, time = {t_num:.6f} s")
            lines.append("Four-step FOC:")
            lines.append(f"  l* = {l_opt_foc:.6f}, U = {u_opt_foc:.6f}, time = {t_foc:.6f} s")
            lines.append("FOC candidates (label, ℓ, U):")
            if len(candidates) == 0:
                lines.append("  None found (defaults may apply)")
            else:
                for c in candidates:
                    lines.append(f"  {c[0]}: ℓ={c[1]:.6f}, U={c[2]:.6f}")
            ax.text(0.01, 0.01, "\n".join(lines), fontsize=10, va='bottom')

            plt.tight_layout(rect=[0, 0, 1, 0.96])
            plt.show()

    # save results to disk for reproducibility
    with open(save_path, 'wb') as f:
        pickle.dump(results, f)

    return results

# -------------------------
# Execute Task 3.1
# -------------------------
if __name__ == '__main__':
    # pi values requested in the assignment for Task 3.1
    pi_values = [1.0, 1.175, 1.5]

    # run analysis and save results in the local folder
    results = task_3_1_analysis(
        pi_values,
        save_path='task3_1_results.pkl',   # changed here
        show_plots=True
    )

    # Print concise summary table to console
    print("\nSummary table (pi, l_num, U_num, time_num, l_foc, U_foc, time_foc, chosen_candidate):")
    for pi in pi_values:
        info = results[pi]
        cand = info['chosen_detail']
        print(f"{pi:.3f}  {info['l_opt_num']:.6f}  {info['u_opt_num']:.6f}  {info['t_num']:.6f}  "
              f"{info['l_opt_foc']:.6f}  {info['u_opt_foc']:.6f}  {info['t_foc']:.6f}  {cand}")

    print("\nThe detailed plots for each pi value have been displayed during execution.")


## Opgave 3.2

In [ ]:
# -----------------------------
# Minimal Task 3.2 cell (run after Task 3.1)
# -----------------------------

pi_grid = np.linspace(0.5, 3.0, 251)

l_star_list = np.empty_like(pi_grid)
c_star_list = np.empty_like(pi_grid)
labels = np.empty_like(pi_grid, dtype=object)

for i, pi in enumerate(pi_grid):
    
    worker = Worker(pi)

    # IMPORTANT: your function returns 5 items → unpack all 5
    l_star, u_star, t_foc, candidates, chosen = find_opt_foc_four_step(worker)

    # Store optimal labor and consumption
    l_star_list[i] = l_star
    c_star_list[i] = worker.income(l_star)

    # Store which region won: 'b', 'k', 'a', or 'none'
    labels[i] = chosen[0]

# Compute region proportions
unique_labels, counts = np.unique(labels, return_counts=True)
proportions = {lab: counts[j] / len(pi_grid) for j, lab in enumerate(unique_labels)}

print("Task 3.2 — proportions of workers choosing each region:")
for lab in ["b", "k", "a", "none"]:
    print(f"  {lab:>4s} : {proportions.get(lab,0)*100:5.2f}%")

# Plot ℓ*(π) and c(π)
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].plot(pi_grid, l_star_list)
ax[0].set_xlabel("Productivity π")
ax[0].set_ylabel("Optimal labor ℓ*(π)")
ax[0].set_title("Labor supply function ℓ*(π)")

ax[1].plot(pi_grid, c_star_list)
ax[1].set_xlabel("Productivity π")
ax[1].set_ylabel("Consumption c(π)")
ax[1].set_title("Implied consumption c(π)")

plt.tight_layout()
plt.show()


## Opgave 3.3

In [ ]:
# -----------------------------------------------------------
# Task 3.3 — Public good / SWF under top tax
#
# ASSUMES the following ALREADY exist from Task 3.1 and 3.2:
#   Worker class, find_opt_foc_four_step, model parameters, etc.
#
# This cell:
#   (1) Computes SWF before and after top tax
#   (2) Plots Lorenz curves before/after top tax
#   (3) Searches over alternative (omega, kappa) pairs to see 
#       whether the SWF can be improved.
#
# -----------------------------------------------------------

# ---- Helper: Compute SWF for given (tau, zeta, omega, kappa) ----
def compute_SWF(N=2000, tau=tau_star, zeta=zeta_star, omega_used=omega, kappa_used=kappa):
    """
    Computes the Social Welfare Function:
        SWF = χ G^η + Σ_i U_i
    where:
        G = total tax revenue
        U_i = worker utility
    Using the assignment calibration:
        χ = 0.5 N
        η = 0.1
        log p ~ N(-0.5 σ_p^2, σ_p^2), σ_p = 0.3
    """
    
    # draw productivities
    sigma_p = 0.3
    pis = np.random.lognormal(mean=-0.5*sigma_p**2, sigma=sigma_p, size=N)
    
    # totals
    total_utility = 0
    total_tax_revenue = 0

    for pi in pis:
        # temporarily override parameters for this worker
        wkr = Worker(pi)
        wkr.tau = tau
        wkr.zeta = zeta
        wkr.omega = omega_used
        wkr.kappa = kappa_used
        wkr.l_k = kappa_used / (w * pi)

        # compute optimal l*
        l_star, u_star, _, _, _ = find_opt_foc_four_step(wkr)

        # compute consumption and income
        c_star = wkr.income(l_star)
        
        # compute taxes: τ*w*pi*l + ω*max(w*pi*l - kappa,0)
        gross = w * pi * l_star
        top_part = max(gross - kappa_used, 0)
        taxes_i = (tau * gross) + (omega_used * top_part) + zeta

        total_tax_revenue += taxes_i
        total_utility += u_star
    
    # government revenue G
    G = total_tax_revenue
    
    # SWF = χ G^η + Σ U_i
    chi = 0.5 * N
    eta = 0.1
    return chi * (G**eta) + total_utility


# -----------------------------------------------------------
# (1) Compare SWF with and without top tax
# -----------------------------------------------------------

# Baseline SWF from optimal linear tax system (omega = 0)
SWF_no_top = compute_SWF(omega_used=0, kappa_used=9999)  # effectively no kink

# SWF under the proposed top tax (given ω=0.2, κ=9)
SWF_with_top = compute_SWF(omega_used=omega, kappa_used=kappa)

print("Task 3.3 (1): Change in SWF when introducing the top tax:")
print(f"  SWF without top tax : {SWF_no_top:.4f}")
print(f"  SWF with top tax    : {SWF_with_top:.4f}")
print(f"  Difference          : {SWF_with_top - SWF_no_top:.4f}")

# -----------------------------------------------------------
# (2) Lorenz curves of consumption before vs after top tax
# -----------------------------------------------------------

# generate workers once for plotting Lorenz curve
N_plot = 2000
sigma_p = 0.3
pis_plot = np.random.lognormal(mean=-0.5*sigma_p**2, sigma=sigma_p, size=N_plot)

def compute_consumption_list(pis, tau, zeta, omega_used, kappa_used):
    c_list = []
    for pi in pis:
        wkr = Worker(pi)
        wkr.tau = tau
        wkr.zeta = zeta
        wkr.omega = omega_used
        wkr.kappa = kappa_used
        wkr.l_k = kappa_used / (w * pi)
        l_star, u_star, _, _, _ = find_opt_foc_four_step(wkr)
        c_list.append(wkr.income(l_star))
    return np.array(c_list)

cons_no_top = compute_consumption_list(pis_plot, tau_star, zeta_star, 0, 9999)
cons_with_top = compute_consumption_list(pis_plot, tau_star, zeta_star, omega, kappa)

def lorenz_curve(values):
    x = np.sort(values)
    cumulative = np.cumsum(x)
    cumulative = cumulative / cumulative[-1]
    population = np.linspace(0, 1, len(values))
    return population, cumulative

p_no, L_no = lorenz_curve(cons_no_top)
p_top, L_top = lorenz_curve(cons_with_top)

plt.figure(figsize=(6, 6))
plt.plot(p_no, L_no, label="No top tax")
plt.plot(p_top, L_top, label="With top tax")
plt.plot([0, 1], [0, 1], 'k--', label="Equality line")
plt.title("Lorenz curve of consumption")
plt.xlabel("Population share")
plt.ylabel("Cumulative consumption share")
plt.legend()
plt.show()

# -----------------------------------------------------------
# (3) Search for (omega, kappa) that improves SWF
# -----------------------------------------------------------

omega_grid = [0.0, 0.1, 0.2, 0.3, 0.4]
kappa_grid = [6, 7, 8, 9, 10, 12]

best = (-1e18, None, None)  # (SWF, omega, kappa)

for om in omega_grid:
    for kp in kappa_grid:
        SWF_val = compute_SWF(omega_used=om, kappa_used=kp)
        if SWF_val > best[0]:
            best = (SWF_val, om, kp)

print("\nTask 3.3 (3): Best (omega, kappa) among tested grid:")
print(f"  Best SWF: {best[0]:.4f}")
print(f"  omega*   = {best[1]}")
print(f"  kappa*   = {best[2]}")
